In [ ]:
# !pip3 install holidays

In [1]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import pandas as pd
import datetime
import numpy as np
import dask.dataframe as dd

In [ ]:
# df = pd.read_csv('/home/faye/data/vehicles_2018_cleaned_dataset')

In [ ]:
# Converting date columns to datetime
# datetime_columns = ['DAYOFSERVICE', 'LASTUPDATE']
# for column in datetime_columns:
#     df[column] = pd.to_datetime(df[column])

In [ ]:
# Getting the Irish holidays - including bank holidays
# https://towardsdatascience.com/5-minute-guide-to-detecting-holidays-in-python-c270f8479387
# irish_holidays_2018 = []
# for date in holidays.Ireland(years=2018).items():
#     irish_holidays_2018.append(str(date[0]))

In [ ]:
# irish_holidays_2018

In [ ]:
# df['Is_HOLIDAY'] = [1 if str(val).split()[0] in irish_holidays_2018 else 0 for val in df['DAYOFSERVICE']]
# df['IS_WEEKDAY'] = [1 if int(val.weekday()) < 5 else 0 for val in df['DAYOFSERVICE']]
# df['DAYOFWEEK'] = [calendar.day_name[val.weekday()] for val in df['DAYOFSERVICE']]
# df['MONTHOFSERVICE'] = df['DAYOFSERVICE'].dt.strftime('%B')

### Calculating how long it took to get to the next stop
As we know, each tripid represents one trip at a certain point of the day. So what we need to do is take away the second stop from the first stop. **actualtime_arr from the second stop - actualtime_dep from the first stop**

**Prequisites:**
* Have each row by specific trip and date.

**Method:**
* Loop through every date and tripid:
    * Initiate an empty list called time_between
    * If tripid and date match in a row:
        * Store PROGRNUMBER in a list then 
        * Loop through the list with PROGRNUMBERS:
            * add to time between: the current progrnumber actualtime_arr MINUS progrnumber previous actualtime_dep
    * Add that populated list to the existing rows in loop
    * Replace rows according to the index
    <br>
    (May cause memory error but whatever we'll try this method)

In [ ]:
leavetimes = pd.read_feather('/home/faye/data/leavetimes_cleaned_2.feather')

In [ ]:
leavetimes['DAYOFSERVICE'] = pd.to_datetime(leavetimes['DAYOFSERVICE'])
leavetimes['DAYOFSERVICE'] = (leavetimes['DAYOFSERVICE'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [ ]:
leavetimes = leavetimes.sort_values(by='PROGRNUMBER')
leavetimes.to_csv('/home/faye/data/leavetimes_cleaned_2.csv', index=True)

In [ ]:
tripid = [list(leavetimes['TRIPID'].unique()), list(leavetimes['DAYOFSERVICE'].unique())]

In [ ]:
# making new column with empty values
leavetimes['TOTALTIME'] = np.nan

In [ ]:
# Going to try and define a function that would return time_between as a list instead
def time_between(prog_num):
    return [0 if stop == 1
           else
           (int(row['ACTUALTIME_ARR'].loc[row['PROGRNUMBER']==stop])) - 
            (int(row['ACTUALTIME_DEP'].loc[row['PROGRNUMBER']==(prog_num[prog_num.index(stop)-1])]))
           for stop in prog_num]

In [ ]:
for id_ in tripid[0]:
    for date in tripid[1]:
        row = leavetimes.loc[(leavetimes['TRIPID']==id_) & (leavetimes['DAYOFSERVICE']==date)]
        row.drop('TOTALTIME', axis=1, inplace=True)
        row['TOTALTIME'] = time_between(list(row['PROGRNUMBER']))
        leavetimes.loc[row.index, 'TOTALTIME'] = row['TOTALTIME']

In [ ]:
leavetimes.isnull().sum()

In [ ]:
leavetimes.shape

# Trying it in Dask

In [ ]:
leavetimes = dd.read_csv('/home/faye/data/leavetimes_cleaned_2.csv')

### I want to find a way to implement what I am trying to do using Dask dataframes

In [ ]:
leavetimes = leavetimes.set_index('Unnamed: 0')

In [ ]:
# Convert to date times
leavetimes['DAYOFSERVICE'] = leavetimes['DAYOFSERVICE'].astype("M8[us]")
# Convert to epoch

In [ ]:
leavetimes

In [ ]:
# Make a list of all of the unique TRIPID and DAYOFSERVICE
tripid = [list(leavetimes['TRIPID'].unique()), list(leavetimes['DAYOFSERVICE'].unique())]

In [ ]:
# Make a new feature named TOTALTIME and only put np.nan values
leavetimes['TOTALTIME'] = np.nan

In [ ]:
# Define a function that will
def time_between(prog_num):
    # return a list comprehension of sliced rows based on the values of PROGRNUMBER's indexes
    return [0 if stop == 1
           else
           (int(row['ACTUALTIME_ARR'].loc[row['PROGRNUMBER']==stop])) - 
            (int(row['ACTUALTIME_DEP'].loc[row['PROGRNUMBER']==(prog_num[prog_num.index(stop)-1])]))
           for stop in prog_num]


In [ ]:
# Loop through the list trips
for id_ in tripid[0]:
    # Loop through list of dates
    for date in tripid[1]:
        # Get rows of matching trips and dates
        row = leavetimes.loc[(leavetimes['TRIPID']==id_) & (leavetimes['DAYOFSERVICE']==date)]
        # Drop TOTALTIME
        row.drop('TOTALTIME', axis=1)
        # Let row totaltime equal to list by calling function
        row['TOTALTIME'] = time_between(list(row['PROGRNUMBER']))
        # Replace the leavetimes rows according to the sub df which has new values for TOTALTIME
        leavetimes.merge(row, left_on='leavetimes', right_index=True)